# Project Code

### ProjectCSGirls 2022

In [1]:
##import libraries for use throughout our code

from IPython.display import clear_output, display, update_display #IPython is the computing shell for our programming!

import smbus #temperature

#temperature and humidity
from bme280 import BME280
bme280 = BME280() 

import busio #air quality sensor
import board #air quality sensor, buzzer, and display

#Air quality sensor
from board import *
i2c = board.I2C() 

#Google Sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

#Adafruit
import adafruit_ccs811
from Adafruit_IO import Client, Feed, RequestError
from dotenv import load_dotenv
import os

import time #for general use

Google Sheets Initial Code - Kristen

In [2]:
sheet_name = "ProjectCSGirls"   #Define the name of our Google Sheet

# use credentials to create a client that interacts with the Google Drive API *FIX THIS @ 2:30!!!*
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('projectcsgirls-345320-ec70b92cf651.json', scopes)
client = gspread.authorize(creds)

# Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).sheet1

APIError: {'errors': [{'domain': 'usageLimits', 'reason': 'accessNotConfigured', 'message': 'Access Not Configured. Drive API has not been used in project 170905669483 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/drive.googleapis.com/overview?project=170905669483 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.', 'extendedHelp': 'https://console.developers.google.com/apis/api/drive.googleapis.com/overview?project=170905669483'}], 'code': 403, 'message': 'Access Not Configured. Drive API has not been used in project 170905669483 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/drive.googleapis.com/overview?project=170905669483 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.'}

Google Sheets Initial Code - Katie

In [6]:
sheet_name = "ProjectCSGirls"   #Define the name of our Google Sheet

# use credentials to create a client that interacts with the Google Drive API *FIX THIS @ 2:30!!!*
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('secret.json', scopes)
client = gspread.authorize(creds)

# Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).sheet1

Testing each measurement individually

In [ ]:
##temperature

## is it working?
## how to print??

def readTemp():
    # By default the address of LM75A is set to 0x48
    address = 0x48

    # Read I2C data and calculate temperature
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

In [ ]:
## humidity
h = bme280.get_humidity()
print (h)

In [ ]:
## CO2 and TVOC
ccs = adafruit_ccs811.CCS811(i2c)
print("CO2: ", ccs.eco2, "TVOC:", ccs.tvoc)

Adafruit Connection - Kristen

In [ ]:
#Load login token from .env file
load_dotenv()

ADAFRUIT_IO_USERNAME = os.environ.get("ADAFRUIT_IO_USERNAME") #Adafruit username
ADAFRUIT_IO_KEY = os.environ.get("ADAFRUIT_IO_KEY") #Adafruit password

aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

Adafruit Connection - Katie

In [1]:
#Load login token from .env file
load_dotenv()

ADAFRUIT_IO_USERNAME = os.environ.get("ADAFRUIT_IO_USERNAME") #Adafruit username
ADAFRUIT_IO_KEY = os.environ.get("ADAFRUIT_IO_KEY") #Adafruit password

aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

Location - Kristen

In [ ]:
metadata = {'lat': 35.92948,
            'lon': -84.1313,
            'ele': 0,
            'created_at':None} 

Location - Katie

In [ ]:
metadata = {"lat": 40.262726,
            "lon": -74.94198,
            "ele": 0,
            "created_at": None}

Reading Adafruit feeds - Katie

In [ ]:
exterFeedB = aio.feeds("external")
humidFeedB = aio.feeds("humidity")
co2FeedB = aio.feeds("co2")
tvocFeedB = aio.feeds("tvoc")

Reading Adafruit feeds - Kristen

In [ ]:
exterFeedM = aio.feeds("externaltemperature")
humidFeedM = aio.feeds("humidity")
co2FeedM = aio.feeds("co2")
tvocFeedM = aio.feesd("tvoc")

Main Loop - Katie

In [ ]:
while true:
    #define letter variables
    tB = bme280.get_temperature()
    eB = tB * (9./5.) +32 
    hB = bme280.get_humidity()
    co2B = ccs.eco2
    tvocB = ccs.tvoc
    
    #send data to Adafruit feeds
    aio.send_data(exterFeed.key, eB, metadata)
    aio.send_data(humidFeed.key, hB, metadata)
    aio.send_data(co2Feed.key, co2B, metadata)
    aio.send_data(tvocFeed.key, tvocB, metadata)
    
    #send and print data on the Google Sheet
    now=datetime.now()
    now_time = now.strftime("%H:%M:%S")
    now_day = now.strftime("%m/%d/%Y")
    row=[now_day, now_time, eB, hB, co2B, tvocB]
    sheet.append_row(row)
    
    #print data in Jupyter Notebooks (below)
    print(eB, ":°Fahrenheit|")
    print(hB, ":% Humidity|")
    print(co2B, ": PPM (CO2)|")
    print(tvocB, ":PPB (TVOC)|")
    print()
    
    #wait a certain amount of time
    time.sleep(60)

Main Loop - Kristen

In [ ]:
while true:
    #define letter variables
    tM = bme280.get_temperature()
    eM = tB * (9./5.) +32 
    hM = bme280.get_humidity()
    co2M = ccs.eco2
    tvocM = ccs.tvoc
    
    #send data to Adafruit feeds
    aio.send_data(exterFeed.key, eM, metadata)
    aio.send_data(humidFeed.key, hM, metadata)
    aio.send_data(co2Feed.key, co2M, metadata)
    aio.send_data(tvocFeed.key, tvocM, metadata)
    
    #send and print data on the Google Sheet
    now=datetime.now()
    now_time = now.strftime("%H:%M:%S")
    now_day = now.strftime("%m/%d/%Y")
    row=[now_day, now_time, eM, hM, co2M, tvocM]
    sheet.append_row(row)
    
    #print data in Jupyter Notebooks (below)
    print(eM, ":°Fahrenheit|")
    print(hM, ":% Humidity|")
    print(co2M, ": PPM (CO2)|")
    print(tvocM, ":PPB (TVOC)|")
    print()
    
    #wait a certain amount of time
    time.sleep(60)